### 2020-07-27T02:12:40Z という形式で出力する方法

https://ufirst.jp/memo/2020/07/27/post-2638/  
2020-07-27T02:12:40Z  
という時刻形式がある。

ISO規格だと ISO 8601  
RFC規格だとRFC 3339 （ほぼ同じと見ていいらしい。Wikipediaの情報だけど）

pandasの時刻データをこの形で作る方法。  
何でか? サンプルデータを作りたくて、その時刻形式がこれだったからです。

後で読む
https://teratail.com/questions/249408


In [1]:
import pandas as pd
import datetime
# pd.options.display.notebook_repr_html = False  # jupyter notebook上での出力形式を制御するために書いています。無くても動きます。

In [2]:
# 動作環境の確認
print(pd.__version__)
import matplotlib
print(matplotlib.__version__)

Duplicate key in file PosixPath('/usr/local/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 258 ('font.family : Hiragino sans')


1.1.2
3.3.1


In [3]:
datetime_list = [
    datetime.datetime(2021, 1, 11),
    datetime.datetime(2021, 2, 11),
    datetime.datetime(2021, 3, 20),
    datetime.datetime(2021, 4, 29),
    datetime.datetime(2021, 5, 5),
]

In [4]:
val_list = [10, 30, 20, 50, 40]

In [5]:
df_datetime = pd.DataFrame({
    'date'    : datetime_list,
    'val' : val_list
})



In [6]:
df_datetime

,date,val
0,2021-01-11,10
1,2021-02-11,30
2,2021-03-20,20
3,2021-04-29,50
4,2021-05-05,40


In [7]:
df_datetime.dtypes

date    datetime64[ns]
val              int64
dtype: object

In [8]:
# dateカラムはタイムゾーンがあるかない→ない（なぜならXXX）
# タイムゾーンを設定します

In [9]:
# https://pandas.pydata.org/docs/reference/api/pandas.Series.tz_localize.html

In [10]:

df_datetime['new'] = df_datetime['date'].tz_localize(tz='UTC')

# これはダメ。インデックスを変換しようとするから。

# Q：何でエラーなの


TypeError: index is not a valid DatetimeIndex or PeriodIndex

In [11]:
# Q ただのtz_localizeと dt.tz_localize って何が違うの
# Q dtって何

# https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.tz_localize.html#pandas.Series.dt.tz_localize
df_datetime['new'] = df_datetime['date'].dt.tz_localize(tz='UTC')


In [12]:
df_datetime

,date,val,new
0,2021-01-11,10,2021-01-11 00:00:00+00:00
1,2021-02-11,30,2021-02-11 00:00:00+00:00
2,2021-03-20,20,2021-03-20 00:00:00+00:00
3,2021-04-29,50,2021-04-29 00:00:00+00:00
4,2021-05-05,40,2021-05-05 00:00:00+00:00


In [13]:
# 2021-01-11 00:00:00+00:00 となってしまった。
# 2021-01-11T00:00:00Z が良いんだけど。

In [14]:
df_datetime.dtypes

date         datetime64[ns]
val                   int64
new     datetime64[ns, UTC]
dtype: object

In [15]:
# Q 時刻を表示する形式を変更する方法はあるのか?
# A 多分ない

# Q ではどうすれば所望の文字列を得られるのか?
# Q typeはどうなってるのか?